In [1]:
from datasets import load_dataset
import transformers
import pandas as pd
import numpy as np
import os
import time

In [2]:
import torch
torch.cuda.is_available()
torch.cuda.empty_cache()

In [3]:
data_path = "./dataset/csv/"

## data load

In [4]:
all_data = load_dataset(
    'csv',
    data_files=os.path.join(
        data_path,
        'order_speech_ko_ppt.csv' ## 4까지 완료
    ),
    sep='\t',
    split='train'
)

Using custom data configuration default-2cf0bbc2a35b5d09


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to C:\Users\AI_server\.cache\huggingface\datasets\csv\default-2cf0bbc2a35b5d09\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


## data cleaning

In [5]:
import re
from unicode import split_syllables, join_jamos
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    batch["text"] = split_syllables(batch["text"])
    return batch

In [6]:
remove_spectial_char_data = all_data.map(remove_special_characters)

  0%|          | 0/100 [00:00<?, ?ex/s]

## Create Wav2Vec2CTCTokenizer

In [7]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab_jamos.json",
                                 unk_token="<unk>",
                                 pad_token="<pad>",
                                 word_delimiter_token="|")

## Create XLSR-Wav2Vec2 Feature Extractor

In [8]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1,
                                             sampling_rate=16000,
                                             padding_value=0.0,
                                             do_normalize=True,
                                             return_attention_mask=True)

In [9]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor,
                              tokenizer=tokenizer)

In [10]:
# from google.colab import drive
# drive.mount('/content/gdrive/')

In [11]:
# processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-ready")

## add audio array

In [12]:
import librosa

In [13]:
sr = 16000

In [14]:
def load_audio(batch):
#     batch['array'],_ = librosa.load('./dataset/audio/'+batch['filename'],sr=16000)
    batch['array'] = np.array(batch['array'][1:-1].split(',')).astype(np.float32)
    return batch

In [15]:
audio_data = remove_spectial_char_data.map(load_audio)

  0%|          | 0/100 [00:00<?, ?ex/s]

In [16]:
type(audio_data[0]['array'])

list

In [17]:
audio_data[0]['array']

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.2546227620181816e-13,
 -2.629366776391029e-12,
 1.9561086691405327e-12,
 -1.0283167546643046e-11,
 8.514041034690667e-12,
 -2.4210945748226465e-11,
 2.9738985579674804e-11,
 -4.436709680200046e-11,
 6.649177042294951e-11,
 -9.122887861812146e-11,
 1.238807112446949e-10,
 -1.6133586133726396e-10,
 1.923707027007282e-10,
 -1.9670189088660806e-10,
 1.669526045189329e-10,
 -4.015029628212652e-11,
 -2.1462671917493736e-10,
 7.055963591184877e-10,
 -1.4651054813796804e-09,
 2.642754

## Data augmetation(RIR applied)

In [18]:
rir_raw,_ = librosa.load('./room_component.wav',sr)
rir = torch.from_numpy(rir_raw.reshape(1,-1))
print(rir.shape)

torch.Size([1, 4160])


In [19]:
def rir_applied(batch):

    speech = torch.from_numpy(np.array(batch['array'],dtype=np.float32).reshape(1,-1))

    speech_ = torch.nn.functional.pad(speech, (rir.shape[1] - 1, 0))
#     print(speech.dtype)
#     print(speech_.dtype)
#     print(rir.dtype)
    augmented = torch.nn.functional.conv1d(speech_[None, ...], rir[None, ...])[0]
    batch['array'] = augmented.reshape(-1)
    return batch

def fast_stretching(batch):
    array = np.array(batch['array'],dtype=np.float32)
    batch['array'] = librosa.effects.time_stretch(array,0.8)
    return batch

def too_fast_stretching(batch):
    array = np.array(batch['array'],dtype=np.float32)
    batch['array'] = librosa.effects.time_stretch(array,0.5)
    return batch

def slow_stretching(batch):
    array = np.array(batch['array'],dtype=np.float32)
    batch['array'] = librosa.effects.time_stretch(array,1.2)
    return batch

In [20]:
rir_applied_audio_data = audio_data.map(rir_applied)

  0%|          | 0/100 [00:00<?, ?ex/s]

In [21]:
fast_stretching_data = audio_data.map(fast_stretching)

  0%|          | 0/100 [00:00<?, ?ex/s]

In [22]:
too_fast_stretching_data = audio_data.map(too_fast_stretching)

  0%|          | 0/100 [00:00<?, ?ex/s]

## Preprocess Data

In [23]:
def prepare_dataset(batch):

    # batched output is "un-batched"
    batch["input_values"] = processor(batch["array"], sampling_rate=16000).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [24]:
ds_list = [
    audio_data,
    rir_applied_audio_data,
    fast_stretching_data,
    too_fast_stretching_data
]
prepare_ds_list = []
for ds in ds_list:
    prepare_ds_list.append(ds.map(
        prepare_dataset,
        remove_columns=ds.column_names,
        # num_proc=2
    ))

  0%|          | 0/100 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ex/s]

In [25]:
augmented_data = torch.utils.data.ConcatDataset(prepare_ds_list)

# Training

## Set-up Trainer

In [26]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.cuda.HalfTensor]]]) -> Dict[str, torch.cuda.HalfTensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [27]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [28]:
from datasets import load_dataset, load_metric, Audio

wer_metric = load_metric("wer")

In [29]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

## Import Model

In [30]:
output_dir="./wav2vec2-large-xlsr-ko-demo"
model_list = os.listdir(output_dir)
model_list[0]

'checkpoint-3000'

In [31]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    os.path.join(output_dir,model_list[0]), 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

XLSR-Wav2Vec2의 첫 번째 구성 요소는 원시 음성 신호에서 음향적으로 의미가 있지만 문맥적으로 독립적인 기능을 추출하는 데 사용되는 CNN 계층 스택으로 구성됩니다.  
모델의 이 부분은 사전 교육 중에 이미 충분히 훈련되었으며 논문에 명시된 바와 같이 더 이상 미세 조정할 필요가 없습니다. 따라서 특징 추출 부분의 모든 파라미터에 대해 require_grad를 False로 설정할 수 있다.

In [32]:
model.freeze_feature_extractor()

C:\Users\AI_server\Anaconda3\envs\STT\lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:1677: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


메모리를 절약하기 위해 그라데이션 체크포인팅을 활성화

In [33]:
model.gradient_checkpointing_enable()

## TrainingArguments

In [34]:
# !pip install accelerate

In [35]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-ko-demo",
  output_dir=output_dir,
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=20,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
#   auto_find_batch_size=True, # need -> pip install accelerate
  load_best_model_at_end=True
)

In [36]:
from transformers import Trainer
from torch.utils.data import random_split

ds_size = len(augmented_data)
train_size = int(ds_size*0.8)
val_size = ds_size - train_size
train_ds, val_ds = random_split(augmented_data,[train_size,val_size])

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


## Model Training

In [37]:
torch.cuda.empty_cache()
# display(torch.cuda.memory_stats("cuda"))
torch.cuda.reset_max_memory_allocated("cuda")
torch.cuda.reset_max_memory_cached("cuda")

print("max_memory_allocated :",
      torch.cuda.max_memory_allocated("cuda"))
print("max_memory_reserved :",
      torch.cuda.max_memory_reserved("cuda"))

max_memory_allocated : 1262210048
max_memory_reserved : 1264582656


C:\Users\AI_server\Anaconda3\envs\STT\lib\site-packages\torch\cuda\memory.py:274: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  FutureWarning)
C:\Users\AI_server\Anaconda3\envs\STT\lib\site-packages\torch\cuda\memory.py:300: FutureWarning: torch.cuda.reset_max_memory_cached now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  FutureWarning)


In [38]:
# torch.cuda.empty_cache()
# torch.cuda.memory_allocated(device="cuda")

In [39]:
trainer.train()

C:\Users\AI_server\Anaconda3\envs\STT\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 320
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 400


Step,Training Loss,Validation Loss,Wer
100,0.166300,0.047755,0.233333
200,0.049100,0.016672,0.120000
300,0.043900,0.033131,0.146667
400,0.039100,0.028646,0.340000


***** Running Evaluation *****
  Num examples = 80
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-ko-demo\checkpoint-100
Configuration saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-100\config.json
Model weights saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-100\pytorch_model.bin
Feature extractor saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-100\preprocessor_config.json
Deleting older checkpoint [wav2vec2-large-xlsr-ko-demo\checkpoint-3000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 80
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-ko-demo\checkpoint-200
Configuration saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-200\config.json
Model weights saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-200\pytorch_model.bin
Feature extractor saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-200\preprocessor_config.json
Deleting older checkpoint [wav2vec2-large-xlsr-ko-demo\checkpoint-3100] due to args.save_tota

TrainOutput(global_step=400, training_loss=0.4334152438491583, metrics={'train_runtime': 397.4078, 'train_samples_per_second': 16.104, 'train_steps_per_second': 1.007, 'total_flos': 9.023015986227489e+17, 'train_loss': 0.4334152438491583, 'epoch': 20.0})

CTC 손실을 사용하여 더 큰 데이터 세트에서 더 큰 모델을 미세 조정하려면 [여기서](https://github.com/huggingface/transformers/tree/master/examples/pytorch/speech-recognition#connectionist-temporal-classification-without-language-model-ctc-wo-lm) 공식 음성 인식 예를 살펴봐야 한다.